# Resume PDF Parsing
extract information for entities:
1. Name
2. Location
3. Email
4. no.of pages
5. Skills
6. Job Title /Designation
7. Degree (eg. B.Sc., M.Tech, etc)
8. Total Years of Experience

In [3]:
#!pip install PyMuPDF

     --------------------------------------- 11.8/11.8 MB 59.4 MB/s eta 0:00:00


In [11]:
import re
from datetime import datetime

In [63]:
import fitz
#fname = 'Data-Analyst-Resume-2.pdf'
fname='sample_input.pdf'
#fname= 'data-scientist-1559725114.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.get_text())

tx = " ".join(text.split('\n'))
print(tx)

Dushyant Bhatt BI / Big Data/ Azure Hyderabad-Deccan, Telangana, Telangana - Email me on Indeed: indeed.com/r/Dushyant- Bhatt/140749dace5dc26f • 10+ years of Experience in Designing, Development, Administration, Analysis, Management in the Business Intelligence Data warehousing, Client Server Technologies, Web-based Applications, cloud solutions and Databases. • Data warehouse: Data analysis, star/ snow flake schema data modeling and design specific to data warehousing and business intelligence environment. • Database: Experience in database designing, scalability, back-up and recovery, writing and optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  • Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL). • Big Data: Worked Azure data lake store/analytics for big data processing and Azure data factory to schedule U-

## 1. Extract no. of pages

In [400]:
num_pages = doc.page_count
print("Number of pages in the resume:", num_pages)
doc.close()

Number of pages in the resume: 2


## Perform Rule based matching using Spacy
## 2. For skills, email

In [191]:
#!python -m spacy download en_core_web_md

In [1]:
import spacy
from spacy import displacy

In [2]:
nlp = spacy.load('en_core_web_md')

In [3]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [4]:
skills= 'jz_skill_patterns.jsonl'
# add entity ruler to pretrained model
ruler = nlp.add_pipe("entity_ruler",before="ner")

In [5]:
## add skills to model
ruler.from_disk(skills)

In [6]:
# Match custom attribute values with regular expressions for email
pattern = [{"label": "EMAIL", "pattern": [{"TEXT": {"REGEX": "([^@|\s]+@[^@]+\.[^@|\s]+)"}}]}]
#pattern = [{"EMAIL": {"REGEX": "([^@|\s]+@[^@]+\.[^@|\s]+)"}}]
#pattern = [{"_": {"EMAIL": {"REGEX": "([^@|\s]+@[^@]+\.[^@|\s]+)"}}}]
#pattern = [{"TEXT": {"REGEX": "([^@|\s]+@[^@]+\.[^@|\s]+)"}}]

In [7]:
ruler.add_patterns(pattern)

In [409]:
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'entity_ruler',
 'ner']

In [9]:
nlp.to_disk('./custom_model1')

In [10]:
model_dir = "./custom_model1"
nlp = spacy.load(model_dir)

In [14]:
doc1 = nlp(tx)
displacy.render(doc1, style="ent", jupyter=True)

In [16]:
i=0
j=0
dct={}
skills=[]
email=[]
for ent in doc1.ents:
    # Print the entity text and its label
    #print(ent.text, ent.label_)
    if ent.label_ == "PERSON" and i==0:
        dct['Name']= ent.text
        i=i+1
    if ent.label_ == "EMAIL":
        email.append(ent.text)
    if ent.label_ == "GPE" and j==0:
        dct["Location"]= ent.text
        j =j+1
    if ent.label_ == "SKILL" :
        skills.append(ent.text)




In [17]:
dct["skills"]= skills
dct["email"]= email[0]

In [22]:
dct['Location']

'Los Angeles'

In [18]:
print(dct)

{'Name': 'Lindsay Atkinson', 'Location': 'Los Angeles', 'skills': ['Twitter', 'business', 'data modeling', 'design', 'Big Data', 'analytics', 'software', 'design', 'design', 'java', 'data processing', 'business', 'business intelligence', 'business intelligence', 'algorithms', 'linear regression', 'monitoring', 'workflow', 'Data Management', 'Analytics', 'Big Data', 'Data Science', 'Analytics', 'Software', 'Design', 'Data Analysis', 'Data Science', 'Big Data', 'Machine Learning', 'Regression Analysis', 'Business Intelligence', 'Database Design', 'Data Mining', 'Google Analytics', 'Google', 'LANGUAGES'], 'email': 'email@email.com'}


In [70]:
custom_nlp = spacy.load('./model_train1/output/model-best')
doc1 = custom_nlp(" 3 years of experience ")

i = 0
r = {}
degrees_resume = []
years_of_exp_resume = []
for ent in doc1.ents:
    if ent.label_ == "Designation" and i == 0:
            r['Job_title'] = ent.text
            i=i+1
    if ent.label_ == "Degree":
        degrees_resume.append(ent.text)
    if ent.label_ == "Years":
        years_of_exp_resume.append(ent.text)


In [71]:
print(years_of_exp_resume)

['3 years of experience']


In [56]:
model_dir = "./custom_model1"
custom_nlp2 = spacy.load(model_dir)
doc_resume = custom_nlp2(tx)
resume_skills = []
j = 0
dct = {}
for ent in doc_resume.ents:
    if ent.label_ == "SKILL":
        resume_skills.append(ent.text)
    if ent.label_ == "GPE" and j == 0:
        dct["Location"] = ent.text
        j = j+1

In [58]:
print(dct)

{'Location': 'Los Angeles'}


## 3. Total years of experience

In [379]:
import re
from dateutil.parser import parse
from datetime import datetime

def calculate_total_experience(resume_text):
    # Search for patterns indicating the start and end of the employment history section
    start_pattern = r"(?:EMPLOYMENT|WORK EXPERIENCE|PROFESSIONAL EXPERIENCE)"
    end_pattern = r"(?:EDUCATION|ACADEMIC QUALIFICATIONS)"

    # Extract the employment history section
    start_match = re.search(start_pattern, resume_text, re.IGNORECASE)
    end_match = re.search(end_pattern, resume_text, re.IGNORECASE)

    if not start_match or not end_match:
        return 0

    employment_section = resume_text[start_match.end():end_match.start()]

    # Extract individual employment entries
    employment_entries = re.findall(r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{2})\s+\d{4}\b.*?(?=\n\n|\n$|$)", employment_section, re.DOTALL)

    total_experience = 0
    current_date = datetime.now()

    # Calculate the duration for each employment entry
    for entry in employment_entries:
        start_date_match = re.search(r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{2})\s+\d{4}\b", entry)
        end_date_match = None

        if start_date_match:
            start_date = parse(start_date_match.group(), fuzzy=True)
            end_date_match = re.search(r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{2})\s+\d{4}\b", entry[start_date_match.end():])

        if end_date_match:
            end_date = parse(end_date_match.group(), fuzzy=True)
        else:
            end_date = current_date

        # Calculate the duration in years
        duration = (end_date - start_date).days / 365
        total_experience += duration

    return round(total_experience, 2)


In [383]:
import re
from dateutil.parser import parse
from datetime import datetime

def calculate_total_experience(resume_text):
    # Search for patterns indicating the start and end of the employment history section
    start_pattern = r"(?:EMPLOYMENT|WORK EXPERIENCE|PROFESSIONAL EXPERIENCE)"
    end_pattern = r"(?:EDUCATION|ACADEMIC QUALIFICATIONS)"

    # Extract the employment history section
    start_match = re.search(start_pattern, resume_text, re.IGNORECASE)
    end_match = re.search(end_pattern, resume_text, re.IGNORECASE)

    if not start_match or not end_match:
        return 0

    employment_section = resume_text[start_match.end():end_match.start()]

    # Extract individual employment entries
    employment_entries = re.findall(r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{2})\s+\d{4}\b.*?(?=\n\n|\n$|$)", employment_section, re.DOTALL)

    total_experience = 0
    current_date = datetime.now()

    # Calculate the duration for each employment entry
    for entry in employment_entries:
        start_date_match = re.search(r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{2})\s+\d{4}\b", entry)
        end_date_match = re.search(r"(?:Present|\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{4})\b", entry)

        if start_date_match:
            start_date = parse(start_date_match.group(), fuzzy=True)
        else:
            continue

        if end_date_match:
            end_date_str = end_date_match.group()

            if end_date_str.lower() == "present":
                end_date = current_date
            else:
                end_date = parse(end_date_str, fuzzy=True)
        else:
            end_date = current_date

        # Calculate the duration in years
        duration = (end_date - start_date).days / 365
        total_experience += duration

    return round(total_experience, 2)


In [396]:
import re
from dateutil.parser import parse
from datetime import datetime

def calculate_total_experience(resume_text):
    # Search for patterns indicating the start and end of the employment history section
    start_pattern = r"(?:EMPLOYMENT|WORK EXPERIENCE|PROFESSIONAL EXPERIENCE)"
    end_pattern = r"(?:EDUCATION|ACADEMIC QUALIFICATIONS)"

    # Extract the employment history section
    start_match = re.search(start_pattern, resume_text, re.IGNORECASE)
    end_match = re.search(end_pattern, resume_text, re.IGNORECASE)

    if not start_match or not end_match:
        return 0

    employment_section = resume_text[start_match.end():end_match.start()]

    # Extract individual employment entries
    employment_entries = re.findall(r"\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?|\d{2})\s+\d{4}\b.*?(?=\n\n|\n$|$)", employment_section, re.DOTALL)

    total_experience = 0
    current_date = datetime.now()

    # Calculate the duration for each employment entry
    for entry in employment_entries:
        start_date_match = re.search(r"\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?|\d{2})\s+\d{4}\b", entry)
        end_date_match = re.search(r"(?:Present|\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{4})\b", entry)

        if start_date_match:
            start_date = parse(start_date_match.group(), fuzzy=True)
        else:
            continue

        if end_date_match:
            end_date_str = end_date_match.group()

            if end_date_str.lower() == "present":
                end_date = current_date
            else:
                end_date = parse(end_date_str, fuzzy=True)
        else:
            end_date = current_date

        # Calculate the duration in years
        duration = (end_date - start_date).days / 365
        total_experience += duration

    return round(total_experience, 2)

In [397]:
tx = "i am Manthan and i have employment experience expereince from 01 2015 - 01 2017, Dec 2016 to present,  in amazonm education qualification"

In [387]:
#tx = "i am Manthan and i have employment experience expereince of 3 years ,  in amazonm education qualification"

In [398]:
years_of_experience = calculate_total_experience(tx)
print("Total years of work experience:", years_of_experience)

Total years of work experience: 1.92


# Dependency parsing 
## years of expereince in job post

In [78]:
import re

def parse_degrees(resume_text):
    degrees = []
    master_patterns = [
        r"M\.E\.",
        r"M\.Tech",
        r"M\.Sc\.",
        r"M\.S\.",
        r"M\.A\.",
        r"M\.B\.A\.",
        r"M\.Com\.",
        r"M\.Ed\.",
        r"M\.P\.A\.",
        r"M\.S\.W\.",
        r"M\.P\.H\.",
        r"M\.Phil\.",
        r"Master's Degree"
    ]
    bachelor_patterns = [
        r"B\.E\.",
        r"B\.Tech",
        r"B\.Sc\.",
        r"B\.S\.",
        r"B\.A\.",
        r"B\.Com\.",
        r"B\.Ed\.",
        r"B\.B\.A\.",
        r"B\.Arch\.",
        r"B\.Pharm\.",
        r"B\.S\.W\.",
        r"B\.C\.A\."
    ]
    phd_patterns = [
        r"PhD",
        r"Ph\.D\.",
        r"D\.Phil\.",
        r"Doctorate in [A-Za-z\s]+",
        r"Doctor of Philosophy"
    ]
    post_graduate_patterns = [
        r"post-graduate degree"
    ]

    master_pattern = "|".join(master_patterns)
    bachelor_pattern = "|".join(bachelor_patterns)
    phd_pattern = "|".join(phd_patterns)
    post_graduate_pattern = "|".join(post_graduate_patterns)

    master_matches = re.findall(master_pattern, resume_text, flags=re.IGNORECASE)
    bachelor_matches = re.findall(bachelor_pattern, resume_text, flags=re.IGNORECASE)
    phd_matches = re.findall(phd_pattern, resume_text, flags=re.IGNORECASE)
    post_graduate_matches = re.findall(post_graduate_pattern, resume_text, flags=re.IGNORECASE)

    degrees.extend(master_matches)
    degrees.extend(bachelor_matches)
    degrees.extend(phd_matches)
    degrees.extend(post_graduate_matches)

    return degrees


In [79]:
degree_entities = parse_degrees(tx)
print("Degree Entities:", degree_entities)

Degree Entities: []


## Fetch data from Job website (Indeed) Realtime

In [1]:
#!pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import re

In [2]:
# Set the path to the chromedriver executable
chromedriver_path = 'C:\Windows\chromedriver.exe'

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(chromedriver_path)
# Navigate to the Indeed website
driver.get('https://in.indeed.com/')
#driver.get('https://indeed.com/')
#driver.get('https://de.indeed.com/')

C:\Users\manth\AppData\Local\Temp\ipykernel_14256\2527370583.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_path)


In [4]:
# Find the search input elements and location input element
search_input = driver.find_element(By.ID, 'text-input-what')
location_input = driver.find_element(By.ID, 'text-input-where')

# Enter the search query and location
#search_input.send_keys('Data Scientist')
#location_input.send_keys('Mumbai')

search_input.send_keys('Senior Software Engineer')
location_input.send_keys('Telangana')


In [5]:
# Submit the search form
search_form = driver.find_element(By.CLASS_NAME, 'yosegi-InlineWhatWhere-primaryButton')
search_form.submit()

In [6]:
try:
    close_popup = driver.find_element(By.CLASS_NAME,"icl-CloseButton icl-Modal-close")
    close_popup.click()
except:
    pass

In [49]:
#sort by date
date_sort = driver.find_element(By.PARTIAL_LINK_TEXT, 'date')
date_sort.click()


## HTML tags works for "Software Developers"

In [44]:
#let the driver wait 3 seconds to locate the element before exiting out
driver.implicitly_wait(3) 


titles=[]
companies=[]
locations=[]
links =[]
reviews=[]
salaries = []
descriptions=[]

for i in range(0,1):
    
    job_card = driver.find_elements(By.CLASS_NAME,'job_seen_beacon')
    
    for job in job_card:
       
    #.  not all companies have review
        try:
            #review = job.find_element(By.XPATH,'.//span[@class="ratingNumber"]').text
            review = job.find_element(By.XPATH,'.//span[@class="ratingNumber"]').text
        except:
            review = "None"
        reviews.append(review)
   #.   not all positions have salary
        try:
            #salary = job.find_element(By.CLASS_NAME,"attribute_snippet").text
            salary = job.find_element(By.XPATH,'.//div[contains(@class,"css-1ihavw2 eu4oa1w0")]').text
        except:
            salary = "None"
    #.  tells only to look at the element       
        salaries.append(salary)
        
        try:
            #location = job.find_element(By.XPATH,'.//div[contains(@class,"companyLocation")]').text
            location = job.find_element(By.XPATH,'.//div[contains(@class,"css-t4u72d eu4oa1w0")]').text
        except:
            location = "None"
    #.  tells only to look at the element       
        locations.append(location)
        
        try:
            #title  = job.find_element(By.XPATH,'.//h2[@class="jobTitle jobTitle-newJob css-bdjp2m eu4oa1w0"]//a').text
            title = job.find_element(By.XPATH,'.//h2[@class="jobTitle css-1u6tfqq eu4oa1w0"]//a').text
        except:
           # title = job.find_element(By.XPATH,'.//h2[@class="jobTitle jobTitle-newJob css-bdjp2m eu4oa1w0"]//a').get_attribute(name="title")
            title = job.find_element(By.XPATH,'.//h2[@class="jobTitle css-1u6tfqq eu4oa1w0"]//a').get_attribute(name="title")
        titles.append(title)
        
        #links.append(job.find_element(By.XPATH,'.//h2[@class="jobTitle jobTitle-newJob css-bdjp2m eu4oa1w0"]//a').get_attribute(name="href"))
        links.append(job.find_element(By.XPATH,'.//h2[@class="jobTitle css-1u6tfqq eu4oa1w0"]//a').get_attribute(name="href"))
        #companies.append(job.find_element(By.XPATH,'.//span[@class="companyName"]').text)
        companies.append(job.find_element(By.XPATH,'.//span[@class="css-1x7z1ps eu4oa1w0"]').text)
        
    
    #try:
     #   next_page = driver.find_element_by_xpath('//a[@aria-label={}]//span[@class="pn"]'.format(i+2))
      #  next_page.click()

   # except:
       # next_page = driver.find_element_by_xpath('//a[@aria-label="Next"]//span[@class="np"]')
        #next_page.click()
    #except:
        #next_page = driver.find_element_by_xpath('//a[.//span[contains(text(),"Next")]]')
        #next_page.click()
        
    
   # print("Page: {}".format(str(i+2)))

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//h2[@class="jobTitle css-1u6tfqq eu4oa1w0"]//a"}
  (Session info: chrome=119.0.6045.160)
Stacktrace:
	GetHandleVerifier [0x00007FF70EA882B2+55298]
	(No symbol) [0x00007FF70E9F5E02]
	(No symbol) [0x00007FF70E8B05AB]
	(No symbol) [0x00007FF70E8F175C]
	(No symbol) [0x00007FF70E8F18DC]
	(No symbol) [0x00007FF70E8E819C]
	(No symbol) [0x00007FF70E9120EF]
	(No symbol) [0x00007FF70E8E80CF]
	(No symbol) [0x00007FF70E9122C0]
	(No symbol) [0x00007FF70E92AAA4]
	(No symbol) [0x00007FF70E911E83]
	(No symbol) [0x00007FF70E8E670A]
	(No symbol) [0x00007FF70E8E7964]
	GetHandleVerifier [0x00007FF70EE00AAB+3694587]
	GetHandleVerifier [0x00007FF70EE5728E+4048862]
	GetHandleVerifier [0x00007FF70EE4F173+4015811]
	GetHandleVerifier [0x00007FF70EB247D6+695590]
	(No symbol) [0x00007FF70EA00CE8]
	(No symbol) [0x00007FF70E9FCF34]
	(No symbol) [0x00007FF70E9FD062]
	(No symbol) [0x00007FF70E9ED3A3]
	BaseThrea

## HTML tags works for "Software Engineers"

In [56]:
#let the driver wait 3 seconds to locate the element before exiting out
driver.implicitly_wait(3) 
titles=[]
companies=[]
locations=[]
links =[]
reviews=[]
salaries = []
descriptions=[]

for i in range(0,1):
    
    job_card = driver.find_elements(By.CLASS_NAME,'job_seen_beacon')
    
    for job in job_card:
       
    #.  not all companies have review
        try:
            #review = job.find_element(By.XPATH,'.//span[@class="ratingNumber"]').text
            review = job.find_element(By.XPATH,'.//span[@class="css-1h3qsl9 eu4oa1w0"]').text
        except:
            review = "None"
        reviews.append(review)
   #.   not all positions have salary
        try:
            #salary = job.find_element(By.CLASS_NAME,"attribute_snippet").text
            salary = job.find_element(By.XPATH,'.//div[contains(@class,"css-1ihavw2 eu4oa1w0")]').text
        except:
            salary = "None"
    #.  tells only to look at the element       
        salaries.append(salary)
        
        try:
            #location = job.find_element(By.XPATH,'.//div[contains(@class,"companyLocation")]').text
            location = job.find_element(By.XPATH,'.//div[contains(@class,"css-t4u72d eu4oa1w0")]').text
        except:
            location = "None"
    #.  tells only to look at the element       
        locations.append(location)
        
        try:
            #title  = job.find_element(By.XPATH,'.//h2[@class="jobTitle jobTitle-newJob css-bdjp2m eu4oa1w0"]//a').text
            #title = job.find_element(By.XPATH,'.//h2[@class="jobTitle jobTitle-newJob css-1u6tfqq eu4oa1w0"]//a').text
            title = job.find_element(By.XPATH,'.//h2[@class="jobTitle jobTitle-newJob css-1u6tfqq eu4oa1w0"]//a').text
        except:
           # title = job.find_element(By.XPATH,'.//h2[@class="jobTitle jobTitle-newJob css-bdjp2m eu4oa1w0"]//a').get_attribute(name="title")
           # title = job.find_element(By.XPATH,'.//h2[@class="jobTitle jobTitle-newJob css-1u6tfqq eu4oa1w0"]//a').get_attribute(name="title")
        titles.append(title)
        
        #links.append(job.find_element(By.XPATH,'.//h2[@class="jobTitle jobTitle-newJob css-bdjp2m eu4oa1w0"]//a').get_attribute(name="href"))
        links.append(job.find_element(By.XPATH,'.//h2[@class="jobTitle jobTitle-newJob css-1u6tfqq eu4oa1w0"]//a').get_attribute(name="href"))
        #companies.append(job.find_element(By.XPATH,'.//span[@class="companyName"]').text)
        companies.append(job.find_element(By.XPATH,'.//span[@class="css-1x7z1ps eu4oa1w0"]').text)

In [126]:
#job_card = driver.find_elements(By.CLASS_NAME,'job_seen_beacon')
#print(job_card)

In [53]:
descriptions=[]
for link in links:
    
    driver.get(link)
    jd = driver.find_element(By.XPATH,'//div[@id="jobDescriptionText"]').text
    descriptions.append(jd)

In [54]:
df=pd.DataFrame()
df['Title']=titles
df['Company']=companies
df['Location']=locations
df['Link']=links
df['Rating']=reviews
df['Salary']=salaries
df['Description']=descriptions

In [55]:
df.head(16)

,Title,Company,Location,Link,Rating,Salary,Description
0,Senior Software Engineer,Wells Fargo,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=95d0cd6552a27c...,3.7,Full-time,The upcoming tech initiatives for the DTI API ...
1,Software Engineer (Equity Plan Solutions-C2H),insightsoftware,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=422277405d13d1...,None,Temporary,Company Description\n\nInsightsoftware is a gr...
2,Senior Enterprise Software Test Engineer - Tes...,Medtronic,"Nanakramguda, Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=827b42adf4115f...,3.9,None,Careers that Change Lives\n\n\nSUMMARY OF POSI...
3,Senior Software Engineer,Combined Insurance,Telangana,https://in.indeed.com/rc/clk?jk=aaaa2cebdcc93f...,None,Full-time,Description for Internal Candidates\nHand on e...
4,Senior Software Engineer,Chubb,Telangana,https://in.indeed.com/rc/clk?jk=c8eaab3d0004e5...,3.7,Full-time,Description for Internal Candidates\nHand on e...
5,SDET Engineer,Oracle,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=8085333c4c82fe...,3.8,"₹40,000 a month",About the Business Unit\n\nOralce-NetSuite is ...
6,Engineer Sr -PPAT,Qualcomm,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=fcc27fdded0d44...,4.0,Call employer,Company:\nQualcomm India Private Limited\nJob ...
7,INS_Staff,EY,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=20df8d991ab3e9...,3.9,None,"At EY, you’ll have the chance to build a caree..."
8,Senior Frontend Web Developer (I/II),Claims Solutions,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=0a2d6a211aad66...,3.6,Full-time,Company Description\n\nWe help the world see n...
9,Senior Enterprise Software Engineer Test Autom...,Medtronic,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=7fefa18f2c33d5...,3.9,None,Careers that Change Lives\n\n\nSUMMARY OF POSI...


In [34]:
df.head(16)

,Title,Company,Location,Link,Rating,Salary,Description
0,Junior Web Developer/Designer,Mahaveer Soft Solutions,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=0d9b18134d06ba...,None,None,CommunicationSkills: Having Good Communication...
1,Software Engineer,Splunk,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=75bd1ea84e5723...,None,None,Join us as we pursue our ground-breaking new v...
2,Software Developer,BigShyft,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=f9ee60977604a0...,None,None,What You'll do:\nYou will be responsible for d...
3,Developer III - Software Engineering,UST,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=9ba440b4d91cfa...,None,None,Role Proficiency:\nIndependently develops erro...
4,Senior PL/SQL Developer,DataEconomy,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=f0c650b79796ad...,None,None,Job Information\nIndustry\nIT Services\nCity\n...
5,IT.Senior Developer,Citco,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=dbbe20f1f4e9a5...,None,None,T24 Senior Developer\nAbout Citco\nSince the 1...
6,"Software Developer I - ServiceNow, IT Support ...",Bristol Myers Squibb,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=20500542ec2df3...,None,None,Working with Us\nChallenging. Meaningful. Life...
7,Software Developer III,FedEx Logistics MEISA,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=30e6e328a418d0...,None,None,General Summary The Software Developer is a Ja...
8,Software,Tata Consultancy Services,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=23fd9da454b647...,None,None,"Job Description\nDear Candidate,\nGreeting Fro..."
9,Software Developer,Heresy Infotech,"Hyderabad, Telangana",https://in.indeed.com/rc/clk?jk=bfe98b47cd8465...,None,None,Key skills to work at Heresy:\nPassionate in l...


In [34]:
df.head(16)

,Title,Company,Location,Link,Rating,Salary,Description
0,Data scientist - Lead,ICICI Prudential Life,"Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=22c17a4a9934a6...,4.0,None,Roles and responsibilities:\n1. Data processin...
1,Data Scientist with Python and Machine Learnin...,Sperton Global AS,"Navi Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=70fb48434dff0b...,None,None,We are seeking a talented and highly skilled D...
2,Data Science M2,Skillety Technologies,"Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=4ab2503d1a4f80...,None,Full-time,Job Description\nJob Description:\n1.Minimum 3...
3,Senior Solutions Engineer - Big Data,Databricks,"Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=f0bb1678a76313...,None,None,"As a Solutions Engineer (Analytics, AI, Big Da..."
4,Sr. Data Scientist,General Mills,"Remote in Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=f11e4dbc13c282...,3.9,None,:\nIndia is among the top ten priority markets...
5,Asset Management - Data Science and Strategic ...,"JPMorgan Chase Bank, N.A.","Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=937bd9ff339c17...,4.1,Full-time\n+1,J.P. Morgan Asset Management (JPMAM) is commit...
6,Executive Director: Data Science and Client In...,"JPMorgan Chase Bank, N.A.","Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=ca11f024d21b07...,4.1,Full-time\n+1,J.P. Morgan Asset Management (JPMAM) is commit...
7,Business Analytics Manager,Hector & Streak,"Navi Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=417788d173c5d2...,None,"₹60,00,000 a year",Graduate with more than 10 years of progressiv...
8,NLP PRACTITIONER,ADVANCED HEALTH,"Mumbai, Maharashtra",https://in.indeed.com/company/Advanced-Health/...,None,Full-time\n+1,It's a globally interesting and challenging jo...
9,Data Scientist,Homesfy Realty,"Thane, Maharashtra",https://in.indeed.com/rc/clk?jk=3d6f2316271baf...,None,Full-time,Role: Data Scientist\n\nJob brief:-\n\nWe at H...


In [35]:
# cleaning data
df['Description']= df["Description"].str.replace('\n', ' ')
df['Description'] = df['Description'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())  # Strip leading/trailing spaces

In [37]:
df[3:6]

,Title,Company,Location,Link,Rating,Salary,Description
3,Senior Solutions Engineer - Big Data,Databricks,"Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=f0bb1678a76313...,None,None,"As a Solutions Engineer (Analytics, AI, Big Da..."
4,Sr. Data Scientist,General Mills,"Remote in Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=f11e4dbc13c282...,3.9,None,: India is among the top ten priority markets ...
5,Asset Management - Data Science and Strategic ...,"JPMorgan Chase Bank, N.A.","Mumbai, Maharashtra",https://in.indeed.com/rc/clk?jk=937bd9ff339c17...,4.1,Full-time\n+1,J.P. Morgan Asset Management (JPMAM) is commit...


In [149]:
df['Description'][3]

'5 - 10 years B.Sc - Statistics | MS/M.Sc - Statistics SAS- Python- Predictive Modelling 9 lkh ~ 12 lkh Data Scientist – Marketing Analytics Performing data-mining and statistical analysis, including predictive modelling, to develop database-marketing strategies that improve customer acquisition/ retention, enhance customer experience, and maximize customer lifetime contribution Job Responsibilities(JR) : Managing 6 to 8 business lines - Provide end to end analytical support for the given business lines with an objective of increasing marketing efficiencies / customer value Develop predictive models - Predictive model for x-sell / activation / retention / attrition control Models to meet minimum performance benchmark Develop behavioral segments and triggers Solution to meet minimum performance benchmark Regular interaction with business / product / campaigns teams Work jointly with campaigns and business team to identify business problem, discuss analytical approach and explain the sol

In [60]:
# Close the browser
driver.quit()
